In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import Kan_NN
import importlib
importlib.reload(Kan_NN)
import pickle
with open('fried_3_scaler_X.pkl', 'rb') as f:
    scaler_X = pickle.load(f)

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_friedman3
def get_loader(scaler_X, n_samples = 20000):
    # Set the seed for reproducibility
    # Generate the Friedmann dataset
    #X_train, y = make_friedman1(n_samples= n_samples, n_features=in_dim, noise=noise)
    X_train, y = make_friedman3(n_samples= n_samples, noise=0.0)
    y_train = np.expand_dims(y, axis=1)
    X_train = scaler_X.transform(X_train)  

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

    # Create TensorDataset for train and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Set batch size and create DataLoader for training and testing
    batch_size = n_samples
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    return train_loader



In [4]:
@torch.no_grad()
def compute_test_loss(test_loader, model):
    criterion = torch.nn.MSELoss()
    running_loss = 0.
    for batch, target in test_loader:
        outputs = model(batch)
        loss = criterion(target, outputs)
        running_loss += loss.item()
    return running_loss / len(test_loader)

In [5]:
import warnings
warnings.simplefilter("ignore")

In [6]:
import time
import dill
n_samples = 30000
widths = [10] ## Grids
in_dims = [4]
noises = [0.,0.2,0.5,1.]
train_losses = []
test_losses = []
for width in widths:
    for in_dim in in_dims:
        for noise in noises:
            test_loader = get_loader(scaler_X,n_samples =n_samples)
            with open(f"models/Friedmann_1_KAN_spline_{noise}_{in_dim}.dill", "rb") as f:
                model = dill.load(f)
            loss = compute_test_loss(test_loader, model[0])
            print(width, noise, in_dim, loss)

10 0.0 4 0.00728948600590229
10 0.2 4 0.010158841498196125
10 0.5 4 0.08268051594495773
10 1.0 4 0.2678241729736328
